In [3]:
import numpy as np
import pandas as pd
from sklearn import model_selection

## Download CSV
Given that decision trees can be used for classification, I'm going to leave in the '?'s as is throughout this assignment. 

In [185]:
house_votes = pd.read_csv("house-votes-84.csv")
house_votes.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


## Training and Test Data Set

In [48]:
feature_names = ['handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution','physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback','education-spending','superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']
label = ['Class-Name']

# Convert Dataframe to two arrays (features rows, label rows)
X = house_votes[feature_names].values
Y = house_votes['Class Name'].values.reshape(-1,1)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.30, random_state=0)

## Implement Node Class
Decision Tree contains two types of nodes. Decision Nodes and Leaf Nodes. For the decision node, it contains a feature index as well as a threshold for that feature. Left and Right utilized to access child nodes. Instead of two separate nodes, I combined into one Node class where leaf_value is only instantiated once we reach a leaf. All parameters are given a default of None.

In [193]:
class Node():
    def __init__(self, feature_index=None, left=None, right=None, threshold=None, info_gain=None, leaf_value=None):
        self.feature_index = feature_index
        self.left = left
        self.right = right
        self.threshold = threshold
        self.info_gain = info_gain
        self.leaf_value = leaf_value 

## Implement Decision Tree Class

In [209]:
class DecisionTree():
    
    '''Constructor
        min_split_num, represents minimum number of samples in order for split
        max_depth, represents the number of levels allowed in the tree
        root, a Node that is originally set to null. Instantiated when fit() is called
    '''    
    def __init__(self,min_split_num=2, max_depth=2):
        self.root = None
        
        self.min_split_num = min_split_num
        self.max_depth = max_depth
    
    '''Constructs Decision Tree with default current depth as 0'''
    def build_tree(self, data, current_depth=0):
        X,Y = data[:,:-1],data[:,-1]
        
        num_samples,num_features = np.shape(X)
        
        if num_samples >= self.min_split_num and current_depth <= self.max_depth:
            best_split = self.find_best_split(data, num_samples, num_features)
            
            if best_split['info_gain'] > 0:
                left_tree = self.build_tree(best_split['data_left'], current_depth + 1)
    
                right_tree = self.build_tree(best_split['data_right'], current_depth + 1)
    
                #Root/Decision Node
                return Node(best_split['feature_index'], left_tree, right_tree, best_split['threshold'], best_split['info_gain'])
    
        leaf_list = list(Y)
        leaf_val = max(leaf_list, key=leaf_list.count)
        
        return Node(leaf_value=leaf_val)
    
    '''find_best_split loops through all features and all values within those features in
       order to find thresholds with the highest calculated info gain'''
    def find_best_split(self, data, num_samples, num_features):
        split_dict = {}
        max_gain = -10000 # Temporary value, looping and comparing to current max_gain throughout loop
        
        for index in range(num_features):
            values = data[:, index]
            
            # Get all unique values of a given feature
            feature_unique_values = np.unique(values)
            
            for threshold in feature_unique_values:
                # Splits current dataset
                data_left, data_right = self.split(data, index, threshold)
                
                # Check children to ensure they aren't empty
                if len(data_left) > 0 and len(data_right) > 0:
                    
                    # remove label from each dataset
                    parent = data[:,-1]
                    child_left = data_left[:,-1]
                    child_right = data_right[:,-1]
                    
                    check_info_gain = self.get_info_gain(parent, child_left, child_right)
                    
                    if check_info_gain > max_gain: # ensures only a higher gain is set, otherwise loop to next threshold
                        split_dict['feature_index'] = index
                        split_dict['threshold'] = threshold
                        split_dict['data_left'] = data_left
                        split_dict['data_right'] = data_right
                        split_dict['info_gain'] = check_info_gain
                        max_gain = check_info_gain #recursive, must finish all features
        
        return split_dict
    
    '''Splits based on whether the current value of the feature column matches the threshold we are trying to calculate'''
    def split(self, data, index, threshold):
        #Create left branch
        true_rows = []
        false_rows = []
        
        for row in data:
            if row[index] <= threshold:
                true_rows.append(row)
            else:
                false_rows.append(row)
                
        data_left = np.array(true_rows)
        data_right = np.array(false_rows)
        
        return data_left,data_right
    
    '''Uses Gini Impurity calculations on parent and left/right children. Returns calculated gain'''
    def get_info_gain(self, parent, left_child, right_child):
        left_weight = len(left_child) / len(parent)
        right_weight = len(right_child) / len(parent)
        
        sum_gain_children = (left_weight * self.gini(left_child)) + (right_weight * self.gini(right_child))
        gain = self.gini(parent) - sum_gain_children
        
        return gain
    
    def gini(self, parent):
        labels = np.unique(parent)
        gini = 1
        
        for label in labels:
            num_rows_true = parent[parent == label]
            
            p = len(num_rows_true) / len(parent)
            gini -= p**2   
            
        return gini
    
    def fit(self, X, Y):
        # Data in build_tree is one single pd.array so combining them solves this design flaw
        data = np.concatenate((X, Y), axis=1) #Switches label to last column, instead of first
        
        self.root = self.build_tree(data) # set root for entire root
    
    def predict_results(self, X):
        results = []
        for row in X:
            results.append(self.predict(self.root, row))
        return pd.array(results)
    
    def predict(self, tree, x):
        if tree.leaf_value is not None:
            return tree.leaf_value
        
        feature_value = x[tree.feature_index]
        
        if feature_value <= tree.threshold:
            return self.predict(tree.left, x)
        else:
            return self.predict(tree.right, x)
    
    def print_results(self, tree = None, spacing="   "):
        if tree is None:
            tree = self.root
            
        if tree.leaf_value is not None:
            print(tree.leaf_value)
            
        else:
            print(feature_names[tree.feature_index], "is", tree.threshold," : info gain", tree.info_gain)
            
            print("%sleft:" % spacing, end="")
            self.print_results(tree.left, spacing + spacing)
            
            print("%sright:" % spacing, end="")
            self.print_results(tree.right, spacing + spacing)

## Implement Testing
Using the original DecisionTree constructor, the fit() and print_results() methods, we can walk through and create a tree to fit to our training data. We will create it using on our 70% training set data and the test using the other 30% of our test set. 

In [195]:
tree = DecisionTree(3, 3)
tree.fit(X_train, y_train)
tree.print_results()

physician-fee-freeze is n  : info gain 0.3914058900451021
   left:adoption-of-the-budget-resolution is ?  : info gain 0.008033226405719004
      left:anti-satellite-test-ban is n  : info gain 0.21333333333333332
            left:el-salvador-aid is ?  : info gain 0.1111111111111111
                        left:republican
                        right:republican
            right:democrat
      right:adoption-of-the-budget-resolution is n  : info gain 0.0025564953919960526
            left:education-spending is ?  : info gain 0.10208333333333339
                        left:republican
                        right:democrat
            right:democrat
   right:synfuels-corporation-cutback is n  : info gain 0.030085660707317732
      left:export-administration-act-south-africa is ?  : info gain 0.007136555239682527
            left:mx-missile is n  : info gain 0.14222222222222206
                        left:republican
                        right:democrat
            right:republican
    

### SKLearn Accuracy Score

In [196]:
y_pred = tree.predict_results(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9465648854961832

### Test effectiveness of Minimum Splitting and Max Depth of this Decision Tree
Originally, we randomly picked 3 for both the minimum splitting and max depth criteria. Just to confirm, I want to quickly test values of 2,4,5,6 to see the results in how the tree looks and accuracy score.

###### Minimum Splitting & Max Depth: 2     Accuracy-Score: 94.66%

In [197]:
# 2 for Minimum Splitting and Max Depth
tree = DecisionTree(2, 2)
tree.fit(X_train, y_train)
tree.print_results()

physician-fee-freeze is n  : info gain 0.3914058900451021
   left:adoption-of-the-budget-resolution is ?  : info gain 0.008033226405719004
      left:anti-satellite-test-ban is n  : info gain 0.21333333333333332
            left:republican
            right:democrat
      right:adoption-of-the-budget-resolution is n  : info gain 0.0025564953919960526
            left:democrat
            right:democrat
   right:synfuels-corporation-cutback is n  : info gain 0.030085660707317732
      left:export-administration-act-south-africa is ?  : info gain 0.007136555239682527
            left:republican
            right:republican
      right:mx-missile is n  : info gain 0.14911764705882347
            left:republican
            right:democrat


###### Minimum Splitting & Max Depth: 4     Accuracy-Score: 95.41%

In [198]:
tree = DecisionTree(4, 4)
tree.fit(X_train, y_train)
tree.print_results()

physician-fee-freeze is n  : info gain 0.3914058900451021
   left:adoption-of-the-budget-resolution is ?  : info gain 0.008033226405719004
      left:anti-satellite-test-ban is n  : info gain 0.21333333333333332
            left:republican
            right:democrat
      right:adoption-of-the-budget-resolution is n  : info gain 0.0025564953919960526
            left:education-spending is ?  : info gain 0.10208333333333339
                        left:republican
                        right:religious-groups-in-schools is n  : info gain 0.02444444444444438
                                                left:democrat
                                                right:democrat
            right:democrat
   right:synfuels-corporation-cutback is n  : info gain 0.030085660707317732
      left:export-administration-act-south-africa is ?  : info gain 0.007136555239682527
            left:mx-missile is n  : info gain 0.14222222222222206
                        left:adoption-of-the-budget-r

In [199]:
y_pred = tree.predict_results(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9541984732824428

###### Minimum Splitting & Max Depth: 5     Accuracy-Score: 94.65%

In [200]:
tree = DecisionTree(5, 5)
tree.fit(X_train, y_train)
tree.print_results()

physician-fee-freeze is n  : info gain 0.3914058900451021
   left:adoption-of-the-budget-resolution is ?  : info gain 0.008033226405719004
      left:anti-satellite-test-ban is n  : info gain 0.21333333333333332
            left:republican
            right:democrat
      right:adoption-of-the-budget-resolution is n  : info gain 0.0025564953919960526
            left:education-spending is ?  : info gain 0.10208333333333339
                        left:republican
                        right:religious-groups-in-schools is n  : info gain 0.02444444444444438
                                                left:democrat
                                                right:democrat
            right:democrat
   right:synfuels-corporation-cutback is n  : info gain 0.030085660707317732
      left:export-administration-act-south-africa is ?  : info gain 0.007136555239682527
            left:mx-missile is n  : info gain 0.14222222222222206
                        left:adoption-of-the-budget-r

In [201]:
y_pred = tree.predict_results(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9465648854961832

###### Minimum Splitting & Max Depth: 6     Accuracy-Score: 95.42%

In [202]:
tree = DecisionTree(6, 6)
tree.fit(X_train, y_train)
tree.print_results()

physician-fee-freeze is n  : info gain 0.3914058900451021
   left:adoption-of-the-budget-resolution is ?  : info gain 0.008033226405719004
      left:democrat
      right:adoption-of-the-budget-resolution is n  : info gain 0.0025564953919960526
            left:education-spending is ?  : info gain 0.10208333333333339
                        left:republican
                        right:religious-groups-in-schools is n  : info gain 0.02444444444444438
                                                left:democrat
                                                right:democrat
            right:democrat
   right:synfuels-corporation-cutback is n  : info gain 0.030085660707317732
      left:export-administration-act-south-africa is ?  : info gain 0.007136555239682527
            left:mx-missile is n  : info gain 0.14222222222222206
                        left:adoption-of-the-budget-resolution is n  : info gain 0.04938271604938271
                                                left:republi

In [203]:
y_pred = tree.predict_results(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9541984732824428

### Confusion Matrix using SKLearn

In [204]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[76,  3],
       [ 3, 49]], dtype=int64)

In [205]:
accuracy = (76+49)/(76+49+3+3)
accuracy

0.9541984732824428

In [206]:
precision = (76)/(76+3)
precision

0.9620253164556962

In [207]:
sensitivity = (49)/(79+3)
sensitivity

0.5975609756097561

In [208]:
f1_score = 2 * ((precision * sensitivity)/(precision + sensitivity))
f1_score

0.7372067702662576

## Evaluation
The accuracy calculated using sklearn and the sklearn confusion matrix seems misleading. If I had more time, I would like to mess around with the testing using kfolding and other strategies to try to decipher what areas this model is the weakest in. The F1 score indicates a relatively decent precision and sensitivity result. I wish I could've done more and dove deeper into this assignment, however, I will submit at this point despite not having nearly as much quality (non-sklearn) testing done. 